# Cuaderno de fine-tunning al modelo de YOLO v8

En este cuaderno hacemos fine-tunning al modelo de YOLO v8 para poder detectar mariposas, ya que por defecto el modelo de YOLO es capaz de detectar 80 tipos de objetos (libros, telefonos, etc) pero no es capaz de detectar mariposas, para ello hacemos fine-tunning que consiste en entrenar el modelo ya funcional con las imagenes de la nueva clase que queremos detectar que en nuestro caso es *butterfly*.

* El modelo de YOLO esta creado con **PyTorch** por lo que una vez creado el modelo y probado tendremos que pasarlo a **TensorFlowLite** para poder usarlo en el movil.
* El dataset de entrenamiento consiste en la union del dataset de las mariposas a las cuales usando **Roboflow** creamos el conjunto de datos necesario ya que necesitan estar en un formato en concreto para poder hacer el *fine-tunning*. Intentamos no usar **Roboflow** ya que queriamos hacerlo todo desde nuestros ordenadores, pero no nos iba muy bien ya que tampoco tenemos grandes GPUs para usar un modelo de detección que nos diera la imagen con la caja de la detección mas los datos de la detección en un `.txt`, lo probamos a hacer con el modelo de YOLO ya que aunque no detecte la mariposa en si detecta otra cosa pero hacia la caja medianamente bien, pero a la hora de entrar no fueron los mejores resultados, por lo que apostamos por usar la página de **Roboflow** que usando el plan gratuito nos permitio darle 1000 imagenes de mariposas y la página usando el modelo de *Dino* que nos dio las fotos y el archivos de `.txt` con el formato que necesitabamos.
* Al entrenar el modelo con solo las imagenes de mariposas funcionaba bien el resultado pero no era capaz de distinguir una mariposa de una botella, por ello decidimos añadir 200 imágenes de objetos randon sin tener un *label* para que sea capaz de diferenciar entre aquello que es mariposa y aquello que no.

## Preparación del cuaderno

Para hacer esto de la forma mas eficiente y rapida hemos usados nuestros ordenadores personales que tienen un gráfica aparte por lo que a la hora de enterar seran mas rapidos.

Para hacer que el entrenamiento use la GPU necesitamos instalar CUDA en nuestro ordenador.

In [1]:
from ultralytics import YOLO
import torch

# Verificar si hay una GPU disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando el dispositivo: {device}")


Usando el dispositivo: cuda


Una vez sabemos que estamos usando *CUDA* que se refiere a que estamos usando la gráfica vemos las versiones que tenemos.

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Cargar modelo YOLOv8 preentrenado y moverlo a la GPU si está disponible
model = YOLO("yolov8n.pt").to(device)

# Verificar que el modelo está en el dispositivo correcto
print(f"El modelo está en el dispositivo: {next(model.parameters()).device}")

2.6.0+cu118
True
11.8
Usando el dispositivo: cuda
El modelo está en el dispositivo: cuda:0


## Entrenamiento

Una vez comprobado que todo esta como queremos, es decir, usando la GPU para entrenar y el modelo de *yolov8n* empezamos con el entrenamiento.

In [ ]:
# Carga YOLO preentrenado
model = YOLO("yolov8n.pt")  

model.train(
    data="modificado_copy/data.yaml",
    epochs=50,              # Puede ser 50, pero con paciencia para detener antes si no mejora
    imgsz=640,              # Resolución estándar de entrada
    batch=8,                # Si la GPU lo soporta, aumenta para entrenar más rápido
    project="butterfly_detection", # Nombre del proyecto
    name="yolov8_butterfly", # Nombre del modelo guardado
    device=0,               # Usar GPU
    half=True,              # FP16 para acelerar en GPU
    lr0=1e-4,               # Tasa de aprendizaje inicial baja (evita sobreajuste)
    workers=8,              # Paralelización para cargar datos más rápido
    augment=True,           # Data augmentation activado
    freeze=10,              # Congela capas iniciales para mantener conocimiento base
    cos_lr=True,            # Ajuste de la tasa de aprendizaje con coseno
    patience=10,            # Early stopping si no mejora tras 10 épocas
    val=True                # Asegura validación en cada época
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 24.2MB/s]


New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.76  Python-3.11.0 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=modificado_copy/data.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=butterfly_detection, name=yolov8_butterfly, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None

100%|██████████| 5.35M/5.35M [00:00<00:00, 17.9MB/s]


AMP: checks passed 


train: Scanning D:\TFM\Codigo\modificado_copy\train\labels... 800 images, 251 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:02<00:00, 472.06it/s]


...

                   all        200        186      0.925      0.994      0.971      0.823



50 epochs completed in 0.453 hours.
Optimizer stripped from butterfly_detection\yolov8_butterfly\weights\last.pt, 6.2MB
Optimizer stripped from butterfly_detection\yolov8_butterfly\weights\best.pt, 6.2MB

Validating butterfly_detection\yolov8_butterfly\weights\best.pt...
Ultralytics 8.3.76  Python-3.11.0 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:07<00:00,  1.79it/s]


                   all        200        186      0.925      0.989      0.969      0.887
Speed: 0.4ms preprocess, 29.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to butterfly_detection\yolov8_butterfly


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002185F6DD4D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

Ya terminado el entrenamiento comprobamos que tal ha ido.

In [ ]:
model = YOLO("D:/TFM/Codigo/runs/detect/train/weights/best.pt") 
print(model.names)

metrics = model.val(data="modificado/data.yaml")
print(metrics)

{0: 'butterfly'}
Ultralytics 8.3.76  Python-3.11.0 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\TFM\Codigo\modificado\valid\labels.cache... 200 images, 14 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.72it/s]


                   all        200        186      0.925      0.995       0.97      0.813
Speed: 1.2ms preprocess, 11.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021867215990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.

## Prueba

Ahora cargamos el modelo que se ha creado con el *fine-tunning* hecho y probamos con una imagen a ver si devuelve la imagen con el cuadrado alrededor

In [4]:
# Cargar el modelo entrenado
model = YOLO("D:/TFM/Codigo/butterfly_detection/yolov8_butterfly/weights/best.pt")  # Reemplaza con la ruta al archivo de pesos guardado

# Probar el modelo con una imagen de prueba
resultados = model.predict("mariposa.jpg", conf=0.7)  # Reemplaza con la ruta a tu imagen de prueba

# Mostrar los resultados
for resultado in resultados:
    resultado.show()


image 1/1 d:\TFM\Codigo\mariposa.jpg: 480x640 1 butterfly, 88.1ms
Speed: 112.1ms preprocess, 88.1ms inference, 940.8ms postprocess per image at shape (1, 3, 480, 640)


## Exportación del modelo

Ahora tenemos que exportar el modelo desde `.pt` -> `.onnx` -> `.tflite`, ya que el modelo en la API va bien con `.pt` pero para el movil es mucho mas optimo utilizar `.tflite`.

In [ ]:
try:
    model.export(format="onnx", half=False, dynamic=False)
    print("Exportación a ONNX completada. El archivo exportado estará optimizado para inferencia en CPU.")
except Exception as e:
    print("Ocurrió un error durante la exportación del modelo:")
    print(e)

Ultralytics 8.3.76  Python-3.11.0 torch-2.6.0+cu118 CPU (Intel Core(TM) i7-7700 3.60GHz)

PyTorch: starting from 'D:\TFM\Codigo\butterfly_detection\yolov8_butterfly\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" "onnxruntime-gpu" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" "onnxruntime-gpu" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" "onnxruntime-gpu" ' returned non-zero exit status 1.
ONNX: export failure  2.1s: No module named 'onnx'
Ocurrió un error durante la exportación del modelo:
No module named 'onnx'
